In [1]:
!pip install transformers
!pip install underthesea
!pip install tokenizers
!pip install py_vncorenlp
!pip install keras
!pip install pytorch-pretrained-bert pytorch-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.3 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=250f7daea6be3b4f8f51f8b7f2ef7bbd1c4b37fc0b5d227f20ff1ce56109f75d
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
     ━━━━━━━━━━━━━

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import underthesea
import os
import torch
from underthesea import text_normalize

import torch
import torch.nn as nn

import copy

import matplotlib.pyplot as plt
import nltk
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xzf aclImdb_v1.tar.gz

--2023-07-06 08:54:23--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  71.8MB/s    in 1.1s    

2023-07-06 08:54:24 (71.8 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    # stop_words = set(stopwords.words('english'))

    # tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

def padding(text, max_len):
    words = text.split(' ')
    if len(words) < max_len:
        words = words + ["<pad>" for _ in range(max_len - len(words))]
    else:
        words = words[0: max_len]

    result = " ".join(words)
    return result

def corpus_length_statistic(corpus):
    lengths = [len(doc) for (doc, _) in corpus]
    min_length = min(lengths)
    max_length = max(lengths)
    avg_length = sum(lengths) / len(lengths)
    total_documents = len(corpus)

    print("Minimum length:", min_length)
    print("Maximum length:", max_length)
    print("Average length:", avg_length)
    print("Total documents:", total_documents)

    return min_length, max_length, avg_length, total_documents

In [5]:
def make_data(path_folder, label):

    result = []

    for filename in os.listdir(path_folder):
        if filename.endswith(".txt"):

            file_path = path_folder + "/" + filename
            with open(file_path, "r") as file:
                file_contents = file.read()
                file_contents = preprocess_text(file_contents)
                file_contents = "<sos> " + file_contents + " <eos>"
                # file_contents = padding(file_contents, max_len=1024)
                result.append(file_contents)
    return result, [label for _ in range(len(result))]


In [6]:
def IMDB_dataset():
    # http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    # tar -xzf aclImdb_v1.tar.gz
    vocabulary_size = 10000
    path_folder_train_pos = "./aclImdb/train/pos"
    path_folder_train_neg = "./aclImdb/train/neg"

    path_folder_test_pos = "./aclImdb/test/pos"
    path_folder_test_neg = "./aclImdb/test/neg"

    train_sentence_pos, train_label_pos = make_data(path_folder_train_pos, 1)
    train_sentence_neg, train_label_neg = make_data(path_folder_train_neg, 0)

    test_sentence_pos, test_label_pos = make_data(path_folder_test_pos, 1)
    test_sentance_neg, test_label_neg = make_data(path_folder_test_neg, 0)

    train_dataset =  train_sentence_pos + train_sentence_neg
    train_label = train_label_pos + train_label_neg

    test_dataset = test_sentence_pos + test_sentance_neg
    test_label = test_label_pos + test_label_neg

    return train_dataset, train_label, test_dataset, test_label

In [7]:
train_dataset, train_label, test_dataset, test_label = IMDB_dataset()

In [8]:
print(train_dataset[0])
print(train_dataset[1])

<sos> best of the zorro serial and one of my favorite serial period this is a period serial set right after the birth of mexico the new nation is counting on the gold produced by this one town to keep the republic solvent however a gold god don del oro is stirring up the indian and stealing the gold for himself it zorro and his band of men to the rescue reed hadley is a winning zorro and he cut a dashing figure a he get into a nice selection of scrap most all of which were reused by the later zorro serial a well a other serial a wellthe story move and it nicely not clear who the real bad guy is there is a reason that ive seen this the most of any serial ive seen it simply a great action adventure film the only thing i can compare it to is the mark of zorro with tyrone power or one of the other swashbuckler of the period it super and highly recommended <eos>
<sos> if you still remember that summer when you had your first kiss first boygirlfriend or first puppy love flingthis film is for

In [9]:
from transformers import BertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(model_name)

In [10]:
def word_2_vec(datasets, tokenizer, max_len):

    tensor_train_dataset = []
    tensor_masked_train = []
    for datapoint in datasets:

        encode = tokenizer.encode_plus(  datapoint,
                                    padding='max_length',
                                    truncation=True,
                                    add_special_tokens=True,
                                    max_length = max_len,
                                    return_tensors='np',
                                    return_token_type_ids=True,
                                    return_attention_mask=True
        )
        encoded_sent = encode['input_ids'].flatten()
        mask =  encode['attention_mask'].flatten()
        tensor_masked_train.append(mask)
        tensor_train_dataset.append(encoded_sent)


    tensor_train_dataset = torch.tensor(tensor_train_dataset)
    tensor_masked_train = torch.tensor(tensor_masked_train)
    return tensor_train_dataset, tensor_masked_train

In [11]:
tensor_train_dataset, tensor_masked_train = word_2_vec(train_dataset, tokenizer, 256)

<ipython-input-10-d9641414b18b>:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  tensor_train_dataset = torch.tensor(tensor_train_dataset)


In [12]:
tensor_test_dataset, tensor_masked_test = word_2_vec(test_dataset, tokenizer, 256)

In [13]:
print(tensor_train_dataset[0])
print(tensor_train_dataset[1])

tensor([  101,   133,  1177,  1116,   135,  1436,  1104,  1103,   195,  1766,
         2180,  7838,  1105,  1141,  1104,  1139,  5095,  7838,  1669,  1142,
         1110,   170,  1669,  7838,  1383,  1268,  1170,  1103,  3485,  1104,
         1143,  8745,  2528,  1103,  1207,  3790,  1110, 11128,  1113,  1103,
         2284,  1666,  1118,  1142,  1141,  1411,  1106,  1712,  1103, 13911,
        27624,  1649,   170,  2284,  5540,  1274,  3687,  1137,  1186,  1110,
        20329,  1146,  1103,  1107, 10359,  1105, 11569,  1103,  2284,  1111,
         1471,  1122,   195,  1766,  2180,  1105,  1117,  1467,  1104,  1441,
         1106,  1103,  5431,  1231,  1174,  1125,  1926,  1110,   170,  2183,
          195,  1766,  2180,  1105,  1119,  2195,   170, 16605,  1158,  2482,
          170,  1119,  1243,  1154,   170,  3505,  4557,  1104, 16720,  1211,
         1155,  1104,  1134,  1127,  1231, 11031,  1118,  1103,  1224,   195,
         1766,  2180,  7838,   170,  1218,   170,  1168,  7838, 

In [14]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [15]:
from torch.utils.data import TensorDataset, DataLoader
batch_size = 32
dataset_train = TensorDataset(tensor_train_dataset, torch.tensor(train_label), tensor_masked_train)

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = TensorDataset(tensor_test_dataset, torch.tensor(test_label), tensor_masked_test)

dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

from transformers import BertModel

model_bert = BertModel.from_pretrained('bert-base-cased')

# print(model_bert)
num_params = sum(p.numel() for p in model_bert.parameters())
print("Number of parameters in bert-base-cased:", num_params)
for i, param in enumerate(model_bert.parameters()):
    if i < 8:
        param.requires_grad = True
    else:
        break

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of parameters in bert-base-cased: 108310272


In [16]:
import torch.nn.functional as F
class MyModel(nn.Module):
    def __init__(self, output_size, drop_out):
        super(MyModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.output_size = output_size
        self.relu = nn.ReLU()
        self.drop_out = nn.Dropout(drop_out)
        self.linear = nn.Linear(in_features=768, out_features=self.output_size, bias=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, batch_input, batch_masked):
        out = self.bert(input_ids=batch_input, attention_mask=batch_masked)
        out = out.pooler_output
        out = self.relu(out)
        out = self.drop_out(out)
        out = self.linear(out)
        probabilities = self.softmax(out)
        return probabilities

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [18]:
model_finetuning = MyModel(output_size=2, drop_out=0.1)
# model = MLP(input_dim=256, hidden_dim=50, output_dim=2)

import torch.optim as optim

def get_criterion_and_optimizer(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.00001)
    return criterion, optimizer

criterion, optimizer = get_criterion_and_optimizer(model=model_finetuning)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
from tqdm import tqdm
def train_model(model, dataloader, criterion, optimizer, epoch, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    running_loss = 0.0
    for i, (batch_inputs, batch_labels, batch_masks) in tqdm(enumerate(dataloader)):

        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)
        batch_masks = batch_masks.to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, batch_masks)
        loss = criterion(outputs, batch_labels)

        loss.backward()
        optimizer.step()

        batch_inputs = batch_inputs.to('cpu')
        batch_labels = batch_labels.to('cpu')
        batch_masks = batch_masks.to('cpu')

        running_loss += loss.item() * batch_inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    print("Training complete epoch!")

from sklearn.metrics import classification_report
def test_model(model, dataloader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()
    total_loss = 0

    y_true = []
    y_pred = []
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for i, (batch_inputs, batch_labels, batch_masks) in tqdm(enumerate(dataloader)):
            batch_inputs = batch_inputs.to(device)
            batch_labels = batch_labels.to(device)
            batch_masks = batch_masks.to(device)

            outputs = model(batch_inputs, batch_masks)

            loss = criterion(outputs, batch_labels)

            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)

            y_pred = y_pred + predicted.to('cpu').tolist()
            y_true = y_true + batch_labels.to('cpu').tolist()

            batch_inputs = batch_inputs.to('cpu')
            batch_labels = batch_labels.to('cpu')
            batch_masks = batch_masks.to('cpu')
    report = classification_report(y_true, y_pred)
    print(report)

In [ ]:
for batch in dataloader_train:
    batch_inputs, batch_labels, batch_masks = batch
    model_finetuning.to("cpu")
    out = model_finetuning(batch_inputs, batch_masks)
    print(out)
    break

In [20]:
num_epochs = 2
for epoch in range(num_epochs):
    # criterion, optimizer
    train_model(model = model_finetuning, dataloader = dataloader_train, criterion = criterion, optimizer = optimizer, epoch = epoch, num_epochs = num_epochs)
    test_model(model = model_finetuning, dataloader = dataloader_test, criterion = criterion)

782it [17:16,  1.33s/it]


Epoch [1/2], Loss: 0.4630
Training complete epoch!


782it [06:08,  2.12it/s]


              precision    recall  f1-score   support

           0       0.91      0.88      0.90     12500
           1       0.89      0.92      0.90     12500

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000



782it [17:20,  1.33s/it]


Epoch [2/2], Loss: 0.4009
Training complete epoch!


782it [06:09,  2.12it/s]


              precision    recall  f1-score   support

           0       0.89      0.91      0.90     12500
           1       0.91      0.89      0.90     12500

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000



In [ ]:
# del model_finetuning
# torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
class BERT_for_sentiment_large(nn.Module):
    def __init__(self, output_size=2, dropout=0.1):
        super(BERT_for_sentiment_large, self).__init__()
        self.output_size = output_size
        self.dropout = dropout
        self.bert =  AutoModel.from_pretrained("vinai/phobert-base", num_labels=2, output_hidden_states=True)
        self.fc1 = nn.Linear(768, 768, bias=True)
        self.layernorm_1 = nn.LayerNorm((768,), eps = 1e-05, elementwise_affine=True)
        self.fc2 = nn.Linear(4*768, self.output_size, bias=True)
        nn.init.normal_(self.fc1.weight, std=0.02)
        nn.init.normal_(self.fc1.bias, 0)
        nn.init.normal_(self.fc2.weight, std=0.02)
        nn.init.normal_(self.fc2.bias, 0)

    def forward(self, inputs, attention_mask):
        outputs = self.bert(input_ids=inputs, attention_mask=attention_mask)
        x = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
        x = self.fc2(x)
        out = x
        return out

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def purify(text):
    vietnamese_characters = "a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾưăạảấầẩẫậắằẳẵặẹẻẽềềểếỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳýỵỷỹ"
    text = text.lower()
    text = text.strip()
    # text = " "+ text + " "
    text = re.sub("[+]", ' ', text) #all math-operating characters removed
    text = re.sub(" \\[a-z]", ' ', text)# all newline and slash characters removed

    text = re.sub('[^' + vietnamese_characters + ']+', ' ', text) # all special characters removed
    text = re.sub("([A-Za-z]+[0-9]+)|([0-9]+[A-Za-z]+)", " ", text) #150g
    text = re.sub("[0-9]{3,}", " ", text) #all too long numbers (>= 3 digits) removed
    text = re.sub(r'(\D)\1{2,}', r'\1', text) # all consecutive duplicated characters removed
    text = text.strip()

    #all single-charater words removed
    tmp_text = []
    for x in text.split(" "):
        if(len(x) > 1):
            tmp_text.append(x)

    text = " ".join(tmp_text)
    text = text_normalize(text)
    return text

In [ ]:
def preprocess(purified_sents, max_len=None):
    preprocessed_sents = []
    for purified_text in purified_sents:

        text = word_tokenize(purified_text, format = "text")
        tmp_text = text
        if(max_len is not None):
            words = tmp_text.split(" ")
            if(len(words) > max_len):
                words = words[0:max_len]
            tmp_text = " ".join(words)

    preprocessed_sents.append(tmp_text)
    return preprocessed_sents

In [ ]:
def encode_plus(preprocessed_sents,max_len=256):
    encoded_sents = []
    masks = []
    for sent in preprocessed_sents:
        sent_info = tokenizer.encode_plus(sent,
                                        padding='max_length',
                                        truncation=True,
                                        add_special_tokens=True,
                                        max_length = max_len,
                                        return_token_type_ids=False,
                                        return_attention_mask=True,
                                        return_tensors='np')
        encoded_sent = sent_info['input_ids'].flatten()
        mask =  sent_info['attention_mask'].flatten()
        encoded_sents.append(encoded_sent)
        masks.append(mask)
    return encoded_sents, masks

In [ ]:
data_full = pd.read_csv("/content/full_train.csv")

In [ ]:
data_full

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
0,0,3839333,10106093.0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",['https://images.foody.vn/res/g97/966781/s800/...,1.0
1,1,2824877,786914.0,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,['https://images.foody.vn/res/g69/688413/s800/...,0.0
2,2,9816702,22467889.0,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",['https://images.foody.vn/res/g72/715078/s800/...,1.0
3,3,2684585,1889449.0,Em có đọc review thấy mng bảo trà sữa nướng đề...,['https://images.foody.vn/res/g90/895545/s800/...,0.0
4,4,2737987,8839942.0,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",['https://images.foody.vn/res/g4/30186/s800/fo...,1.0
...,...,...,...,...,...,...
9068,9066,11236510,23044142.0,Thực sự mà nói thấy mọi người đánh giá nhiều q...,['https://images.foody.vn/res/g94/930433/s800/...,1.0
9069,9067,1831645,10117823.0,👉👉👉LẨU THÁI TÔMYUM\n👉👉Đc: 19 Lò Đúc-HBT-HN\n👉T...,['https://images.foody.vn/res/g16/150633/s800/...,1.0
9070,9068,2155495,11636069.0,Ngay từ lúc đầu tiên bước vào nhà hàng đã được...,['https://images.foody.vn/res/g74/736118/s800/...,1.0
9071,9069,7357032,12208284.0,Đặt ăn thử mà thấy ngón cá...! 🤩 sẽ còn ủn hộ ...,['https://images.foody.vn/res/g104/1036659/s80...,1.0


In [ ]:
data_infer = [purify(str(train_sent)) for train_sent in data_full['Comment']]


In [ ]:
from underthesea import word_tokenize
data_infer_preprocess = preprocess(data_infer)

In [ ]:
ids, masks = encode_plus(data_infer_preprocess, max_len=256)

In [ ]:
tensor_inputs = torch.tensor(ids)
tensor_masks = torch.tensor(masks)
tensor_labels = torch.tensor(data_full['Rating'])

In [ ]:
model_loaded = BERT_for_sentiment_large()

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_loaded.loaded.load_state_dict(torch.load("/content/phobert_linear_train_crawl_3.pt"))